In [1]:
import argparse
import os
import sys
import time
import numpy as np

from PIL import Image

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import urllib
import time

In [2]:
import torch
from functools import partial
from torch import nn, einsum
from einops import repeat, rearrange

In [3]:
def exists(val):
    return val is not Nonev

In [4]:
def map_fn(fn, *args, **kwargs):
    def inner(*arr):
        return map(lambda t: fn(t, *args, **kwargs), arr)
    return inner

In [5]:
def safe_cat_tf(t, el, dim =0):
    if not exists(t):
        return el
    return tf.concat((t, el), axis = dim)

In [6]:
def safe_cat_pt(t, el, dim = 0):
    if not exists(t):
        return el
    return torch.cat((t, el), dim = dim) #concat

# tests

In [7]:
value_dim = 64
key_dim = 64
hidden_dim = 512
output_dim = 4
encoder = None

In [8]:
testA1 = tf.ones((10,2))
testA2 = tf.constant(2.0, shape=(10, 2))
testA1

2022-12-01 01:59:34.148174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-01 01:59:34.148217: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-01 01:59:34.148291: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-la-2dkoenig): /proc/driver/nvidia/version does not exist
2022-12-01 01:59:34.148681: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [9]:
tfC = tf.keras.layers.concatenate([testA1,testA2], axis=-1)
tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.]], dtype=float32)>

In [10]:
tfWk = 2
tfWk * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.]], dtype=float32)>

In [11]:
tf.math.reduce_sum(tfWk * tfC, axis = 1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.], dtype=float32)>

In [12]:
c = tf.constant(1.5, shape=(10, 1))
c * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ]], dtype=float32)>

In [13]:
torchA1 = torch.ones((10,2))
torchA2 = torch.full((10,2),2.0)
torchA1

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [14]:
torchC = torch.cat((torchA1,torchA2), dim=-1)
torchC

tensor([[1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.]])

In [15]:
torchWk = 2
torchWk * torchC

tensor([[2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.]])

In [16]:
(torchWk * torchC).sum(dim = 1)

tensor([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.])

In [17]:
twkt = torch.full((10,1),1.5)
twkt * torchC

tensor([[1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000]])

In [18]:
tlist = []

for i in range(0, 4):
    tlist.append(torch.full((10,1),i, dtype=float))
tlist

[tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]], dtype=torch.float64),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]], dtype=torch.float64),
 tensor([[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]], dtype=torch.float64),
 tensor([[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]], dtype=torch.float64)]

In [19]:
torch.stack(tlist)

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]]], dtype=torch.float64)

In [20]:
tflist = []

for i in range(0,5):
    tflist.append(tf.constant(i, shape=(10, 1), dtype=float))
    
tflist

[<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
      

In [21]:
tf.stack(tflist)

<tf.Tensor: shape=(5, 10, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]],

       [[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]],

       [[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]],

       [[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]], dtype=float32)>

# img test

In [22]:
allI = []
for i in range(3):
    img_fname = '../imgs/' + str(i) + '.png'
    img = torch.Tensor(np.array(Image.open(img_fname))) / 255.
    allI.append(img)
allI = torch.stack(allI, 0)
allI.shape

torch.Size([3, 32, 32])

In [23]:
tfimg = tf.convert_to_tensor(np.array(Image.open(img_fname)) / 255.)
tfimg.shape

TensorShape([32, 32])

# Einsum test

In [26]:
tf_test_1 = np.random.uniform(size=(1,32,32))
tf_test_2 = np.random.uniform(size=(1,32))
tf_test1.shape

NameError: name 'tf_test1' is not defined

In [27]:
sim = tf.einsum('b n d, b d -> b n',tf_test_1, tf_test_2)
sim                                      

<tf.Tensor: shape=(1, 32), dtype=float64, numpy=
array([[10.02943939,  8.14799901,  8.72288361,  7.65564583,  7.98988   ,
         9.76207988,  9.56242772,  9.50701888,  9.40607796, 10.31015884,
         9.07277309,  9.94455877, 10.25403429,  9.11970804, 11.34382705,
        11.40559457,  9.91675205,  9.19788983, 10.86410939, 11.12031193,
         9.96533638,  9.06565231,  9.13012947,  8.45070044,  8.24349484,
        11.01577088, 10.5269025 ,  6.99731903, 10.20250775,  9.6150974 ,
        10.43246785,  7.99250554]])>

In [28]:
torch.einsum('b n d, b d -> b n',
             torch.from_numpy(tf_test_1), 
             torch.from_numpy(tf_test_2))

tensor([[10.0294,  8.1480,  8.7229,  7.6556,  7.9899,  9.7621,  9.5624,  9.5070,
          9.4061, 10.3102,  9.0728,  9.9446, 10.2540,  9.1197, 11.3438, 11.4056,
          9.9168,  9.1979, 10.8641, 11.1203,  9.9653,  9.0657,  9.1301,  8.4507,
          8.2435, 11.0158, 10.5269,  6.9973, 10.2025,  9.6151, 10.4325,  7.9925]],
       dtype=torch.float64)

# TF Array TEst

In [36]:
out = tf.TensorArray(dtype=tf.float32,
                             size=2,
                             dynamic_size=True)

In [37]:
out.size()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [38]:
out.write(0, (1,1))

In [39]:
out.write(1,(2,2))

In [40]:
out.size()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [32]:
#out.stack()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [2., 2.]], dtype=float32)>